In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
import matplotlib.pyplot as plt
import time
import os

from UnarySim.sw.kernel.nn_utils import *

In [3]:
cwd = os.getcwd()
print(cwd)

d:\project\Anaconda3\Lib\site-packages\UnarySim\sw\test\mlp


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
# MNIST data loader
transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root=cwd+'/data/mnist', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = torchvision.datasets.MNIST(root=cwd+'/data/mnist', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, num_workers=4)

In [6]:
layer_width = 16384
total_epoch = min(int(layer_width / 512 * 20), 200)
lr = 0.0001
print(total_epoch)

200


In [7]:
# model = LeNet()
# model = LeNet_clamp()
model = MLP3(layer_width)
# model = MLP3_clamp()
# model = MLP3_clamp_train()
model.to(device)
summary(model, torch.zeros((1, 1, 32, 32)).to(device))

         Kernel Shape Output Shape      Params    Mult-Adds
Layer                                                      
0_fc1   [1024, 16384]   [1, 16384]    16.7936M   16.777216M
1_do1               -   [1, 16384]           -            -
2_fc2  [16384, 16384]   [1, 16384]  268.45184M  268.435456M
3_do2               -   [1, 16384]           -            -
4_fc3     [16384, 10]      [1, 10]     163.85k      163.84k
------------------------------------------------------------
                           Totals
Total params           285.40929M
Trainable params       285.40929M
Non-trainable params          0.0
Mult-Adds             285.376512M


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_fc1,"[1024, 16384]","[1, 16384]",16793600.0,16777216.0
1_do1,-,"[1, 16384]",NaN,NaN
2_fc2,"[16384, 16384]","[1, 16384]",268451840.0,268435456.0
3_do2,-,"[1, 16384]",NaN,NaN
4_fc3,"[16384, 10]","[1, 10]",163850.0,163840.0


In [8]:
bitwidth = 8
clipper = NN_SC_Weight_Clipper(bitwidth=bitwidth)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [10]:
for epoch in range(total_epoch):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
    
    model.eval()
    model.apply(clipper)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Train - Epoch %d, Loss: %f, Test Accuracy: %f %%' \
          % (epoch, loss.detach().cpu().item(), 100 * correct / total))

print('Finished Training')

Train - Epoch 0, Loss: 1.523641, Test Accuracy: 94.940000 %
Train - Epoch 1, Loss: 1.515995, Test Accuracy: 92.360000 %
Train - Epoch 2, Loss: 1.561596, Test Accuracy: 93.310000 %
Train - Epoch 3, Loss: 1.596567, Test Accuracy: 93.250000 %
Train - Epoch 4, Loss: 1.544771, Test Accuracy: 93.450000 %
Train - Epoch 5, Loss: 1.544534, Test Accuracy: 94.040000 %
Train - Epoch 6, Loss: 1.690317, Test Accuracy: 93.150000 %
Train - Epoch 7, Loss: 1.534067, Test Accuracy: 94.050000 %
Train - Epoch 8, Loss: 1.544484, Test Accuracy: 93.900000 %
Train - Epoch 9, Loss: 1.534067, Test Accuracy: 94.330000 %
Train - Epoch 10, Loss: 1.586151, Test Accuracy: 94.410000 %
Train - Epoch 11, Loss: 1.554902, Test Accuracy: 93.780000 %
Train - Epoch 12, Loss: 1.502823, Test Accuracy: 93.860000 %
Train - Epoch 13, Loss: 1.554901, Test Accuracy: 94.130000 %
Train - Epoch 14, Loss: 1.575735, Test Accuracy: 94.210000 %
Train - Epoch 15, Loss: 1.560897, Test Accuracy: 93.940000 %
Train - Epoch 16, Loss: 1.562357, 

Train - Epoch 134, Loss: 1.513234, Test Accuracy: 94.270000 %
Train - Epoch 135, Loss: 1.544484, Test Accuracy: 94.090000 %
Train - Epoch 136, Loss: 1.523651, Test Accuracy: 94.240000 %
Train - Epoch 137, Loss: 1.554862, Test Accuracy: 94.380000 %
Train - Epoch 138, Loss: 1.513234, Test Accuracy: 94.250000 %
Train - Epoch 139, Loss: 1.606984, Test Accuracy: 94.260000 %
Train - Epoch 140, Loss: 1.565673, Test Accuracy: 93.570000 %
Train - Epoch 141, Loss: 1.544484, Test Accuracy: 94.520000 %
Train - Epoch 142, Loss: 1.513234, Test Accuracy: 94.510000 %
Train - Epoch 143, Loss: 1.513234, Test Accuracy: 94.900000 %
Train - Epoch 144, Loss: 1.502817, Test Accuracy: 95.090000 %
Train - Epoch 145, Loss: 1.534067, Test Accuracy: 95.060000 %
Train - Epoch 146, Loss: 1.502817, Test Accuracy: 94.840000 %
Train - Epoch 147, Loss: 1.523651, Test Accuracy: 94.910000 %
Train - Epoch 148, Loss: 1.565317, Test Accuracy: 94.860000 %
Train - Epoch 149, Loss: 1.534067, Test Accuracy: 93.960000 %
Train - 

In [11]:
model_path = cwd+"\saved_model_state_dict"+"_"+str(bitwidth)+"_bitwidth_"+str(layer_width)+"_layerwidth_"+str(lr)+"_lr"
torch.save(model.state_dict(), model_path)

# test load from state_dict

In [12]:
model_path = cwd+"\saved_model_state_dict"+"_"+str(bitwidth)+"_bitwidth_"+str(layer_width)+"_layerwidth_"+str(lr)+"_lr"
model = MLP3(layer_width)
model.load_state_dict(torch.load(model_path))
model.eval()
model.to(device)
# model.apply(clipper)
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 95.000000 %


In [13]:
print(model.fc1.weight.max())
print(model.fc1.weight.min())
print(model.fc2.weight.max())
print(model.fc2.weight.min())
print(model.fc3.weight.max())
print(model.fc3.weight.min())

tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-1., device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.1406, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.1172, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.0859, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.0938, device='cuda:0', grad_fn=<MinBackward1>)
